In [ ]:
#importing necessary libraries
from qiskit import *
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_aer import QasmSimulator
#Constructing the circuit
n = int(input("Enter the length of bitstring: "))
n_q = n
n_c = n
qc = QuantumCircuit(n_q, n_c)
for i in range(n):
    qc.h(i)
    qc.measure(i, i)
qc.draw('mpl')
#Execution of Circuit(for multiple shots to understand superposition)
shot = int(input("Enter the number of shots you want(shots>1): "))
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc, shots=shot)
result_superposition = job.result().get_counts()
result_superposition
plot_histogram(result_superposition)
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
QiskitRuntimeService.save_account(channel="ibm_quantum", token="1cec2291cf5af5ab17f315eb0981c6eba52d9c117f68e17177ac022ed67a7aa27404e4d63a05784e812ba1a90d1834a38f0ffe2a2a8cb0ac001105acc23aad95")

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
 
# Setting options during primitive initialization
estimator = Estimator(backend, options={"resilience_level": 2})
from qiskit.quantum_info import SparsePauliOp

# Define a valid observable using Pauli X operators on each qubit
observable = SparsePauliOp.from_list([("X" * n, 1.0)])  # n Pauli Z's acting on each qubit
print(observable)
#transpiled_qc = transpile(qc, backend=backend)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=1234
)
isa_circuit = pass_manager.run(qc)

isa_circuit.draw("mpl", idle_wires=False, scale=0.7, fold=-1)
isa_observable = observable.apply_layout(isa_circuit.layout)

print("Original observable:")
print(observable)
print()
print("Observable with layout applied:")
print(isa_observable)
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Batch
# To store job results
jobs = []

# Create a batch execution to test different configurations
with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    
    # Set default number of shots to 100,000 for each job
    estimator.options.default_shots = 100000
    
    # Define the observable and circuit pair for the Estimator
    pub = (isa_circuit, isa_observable)

    # Job 0: No error mitigation
    estimator.options.optimization_level = 0  # Disable runtime compilation
    estimator.options.resilience_level = 0    # Disable error mitigation
    job0 = estimator.run([pub])               # Submit job
    jobs.append(job0)


    # Job 1:Gate Twirling
    estimator.options.twirling.enable_gates = True          # Enable gate twirling
    estimator.options.twirling.num_randomizations = "auto"  # Automatic randomization count
    job2 = estimator.run([pub])                             # Submit job with DD + TREX + Twirling
    jobs.append(job2)

    # Job 2:Twirling + Zero-Noise Extrapolation (ZNE)
    estimator.options.resilience.zne_mitigation = True      # Enable ZNE mitigation
    estimator.options.resilience.zne.noise_factors = (1, 3, 5)  # Noise scaling factors
    estimator.options.resilience.zne.extrapolator = ("exponential", "linear")  # Extrapolation methods
    job3 = estimator.run([pub])                             # Submit job with full error mitigation
    jobs.append(job3)
#Post-process and return results in classical format
import matplotlib.pyplot as plt
import numpy as np
# Retrieve the job results
results = [job.result() for job in jobs]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array([float(pub_result.data.evs) for pub_result in pub_results])
standard_errors = np.array([float(pub_result.data.stds) for pub_result in pub_results])

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation", "+ Twirling", "+ ZNE"]
ax.bar(range(len(labels)), expectation_vals, yerr=standard_errors, label="experiment")
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

#We can see there is no significant improvement and Zero Noise Extrapolation has lower performance than no mitigation, so now we execute readout error mitigation technique and compare it with no error and Twirling and see the result
jobs_new = []

# Create a batch execution to test different configurations
with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    
    # Set default number of shots to 100,000 for each job
    estimator.options.default_shots = 100000
    
    # Define the observable and circuit pair for the Estimator
    pub = (isa_circuit, isa_observable)

    # Job 1: No error mitigation
    estimator.options.optimization_level = 0  # Disable runtime compilation
    estimator.options.resilience_level = 0    # Disable error mitigation
    job1 = estimator.run([pub])               # Submit job
    jobs_new.append(job1)

    #Job 2: Readout Error Mitigation (TREX)
    estimator.options.resilience.measure_mitigation = True  # Enable readout error mitigation
    job2 = estimator.run([pub])                             # Submit job with  TREX
    jobs_new.append(job2)
# Job 3:Gate Twirling
    estimator.options.twirling.enable_gates = True          # Enable gate twirling
    estimator.options.twirling.num_randomizations = "auto"  # Automatic randomization count
    job3 = estimator.run([pub])                             # Submit job with TREX + Twirling
    jobs_new.append(job3)


#Post-process and return results in classical format
import matplotlib.pyplot as plt

# Retrieve the job results
results = [job.result() for job in jobs_new]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array([float(pub_result.data.evs) for pub_result in pub_results])
standard_errors = np.array([float(pub_result.data.stds) for pub_result in pub_results])

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation","TREX","+Twirling"]
ax.bar(range(len(labels)), expectation_vals, yerr=standard_errors, label="experiment")
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

In [2]:
import streamlit as st
from qiskit import *
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_aer import QasmSimulator

st.title("Quantum Random Number Generator (QRNG)")

# User inputs
n = st.number_input("Enter the length of bitstring:", min_value=1, value=5)
shots = st.number_input("Enter the number of shots (shots > 1):", min_value=2, value=1024)

# Button to run the QRNG
if st.button("Generate Random Number"):
    # Constructing the circuit
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc.h(i)
        qc.measure(i, i)

    # Execute the circuit
    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc, shots=shots)
    result = job.result().get_counts()

    # Display results
    st.write("Result:", result)
    st.bar_chart(result)

2024-10-18 22:29:41.940 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
 streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py 

SyntaxError: invalid syntax (1917541618.py, line 1)

In [ ]:
# Title for the app
st.title("Quantum Random Number Generator")

# Input field for number of qubits (bitstring length)
n = st.number_input("Enter the length of bitstring (number of qubits):", min_value=1, value=1)

# Create a Quantum Circuit
qc = QuantumCircuit(n, n)

# Apply Hadamard gate to all qubits to create superposition
for i in range(n):
    qc.h(i)
    qc.measure(i, i)

# Display the quantum circuit diagram
#st.write("### Quantum Circuit:")
#st.pyplot(qc.draw('mpl'))

# Run the circuit on a simulator when button is clicked
if st.button("Generate Random Number"):
    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc, shots=shots)
    result = job.result().get_counts()
    
    # Extract the random bitstring and convert to decimal
    random_bitstring = list(result.keys())[0]
    random_number = int(random_bitstring, 2)
    
    # Display the random number
    st.write(f"### The generated random number is: {random_number}")
    
    # Display the measurement outcome (bitstring)
    st.write(f"### Measurement Outcome: {random_bitstring}")

In [ ]:
import streamlit as st
from qiskit import *
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_aer import QasmSimulator

# Title for the app
st.title("Quantum Random Number Generator Using Entanglement")

# Input field for number of qubit pairs
n = st.number_input("Enter the number of entangled qubit pairs:", min_value=1, value=1)

# Create a Quantum Circuit with 2n qubits (for n entangled pairs)
qc = QuantumCircuit(2*n, 2*n)

# Create entanglement for each qubit pair
for i in range(n):
    qc.h(2*i)           # Apply Hadamard gate to the first qubit of the pair
    qc.cx(2*i, 2*i+1)   # Apply CNOT gate to entangle the first and second qubit
    qc.measure(2*i, 2*i)    # Measure first qubit
    qc.measure(2*i+1, 2*i+1) # Measure second qubit



# Run the circuit on a simulator when button is clicked
if st.button("Generate Random Number"):
    simulator = Aer.get_backend('qasm_simulator')
    job = simulator.run(qc, shots=1)
    result = job.result().get_counts()

    # Extract the random bitstring (correlated results due to entanglement)
    random_bitstring = list(result.keys())[0]
    random_number = int(random_bitstring, 2)
    
    # Display the random number
    st.write(f"### The generated random number is: {random_number}")
    
    # Display the measurement outcome (bitstring)
    st.write(f"### The generated random Bitstring is: {random_bitstring}")

In [ ]:
streamlit run entangled_qrng_app.py

In [10]:
from qiskit import *
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_aer import QasmSimulator

n = int(input("Enter the number of entangled qubit pairs: "))
nq = 2*n
nc = 2*n
qc = QuantumCircuit(nq, nc)
for i in range(n):
    qc.h(2*i)           # Apply Hadamard gate to the first qubit of the pair
    qc.cx(2*i, 2*i+1)   # Apply CNOT gate to entangle the first and second qubit
    qc.measure(2*i, 2*i)    # Measure first qubit
    qc.measure(2*i+1, 2*i+1)



Enter the number of entangled qubit pairs:  2


In [34]:
import matplotlib.pyplot as plt
qc.draw('mpl')


<Figure size 621.941x451.5 with 1 Axes>

In [20]:
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc, shots=1)
result = job.result().get_counts()
keys = result.keys()
for key in keys:
    print("The random number is: ", int(key,2))

The random number is:  3


In [26]:
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc, shots=1024)
result_superposition = job.result().get_counts()
plot_histogram(result_superposition)

<Figure size 640x480 with 1 Axes>

In [28]:
result_superposition

{'0011': 250, '0000': 256, '1100': 281, '1111': 237}

In [38]:
def create_quantum_walk_circuit(steps):
    # Create a quantum circuit with 2 qubits: 1 for position and 1 for coin state
    qc1 = QuantumCircuit(2)
    
    # Initialize the coin in superposition
    qc1.h(1)  # Hadamard gate on the coin qubit
    
    for _ in range(steps):
        # Apply the quantum walk step
        qc1.cx(1, 0)  # Control: coin qubit; Target: position qubit
        qc1.h(1)  # Coin flip (Hadamard)
        
    # Measure the position qubit to extract the random number
    qc1.measure_all()
    return qc

# Define the number of steps in the quantum walk
steps = 5  # You can adjust this based on desired randomness

# Create the quantum walk circuit
qc1 = create_quantum_walk_circuit(steps)

# Simulate the quantum walk
simulator = Aer.get_backend('qasm_simulator')
job1 = simulator.run(qc, shots=1024)
result1 = job.result().get_counts()

# Get the counts and visualize
#counts1 = result.get_counts(qc1)
#print("Counts:", counts1)
plot_histogram(result1)

<Figure size 640x480 with 1 Axes>